In [5]:
import pathlib
import contextlib
import re

import requests
import pandas as pd

In [2]:
files_url = 'https://www.eurisportal.eu/AWFImportData/api/ExportFile/GetFiles?countryCode={country_code}'
download_url = 'https://www.eurisportal.eu/AWFImportData/api/ExportFile/DownloadFile?fileName={filename}'

headers = {"Accept": "application/json", 'User-Agent': 'test-agent'}
data_dir = pathlib.Path('~/data/euris').expanduser()
                    
# Rhein (CH, DE), Rhin (FR), Canal d’Alsace (F) Rijn (NL), Waal (NL), Boven Merwede (NL), Beneden Merwede (NL), Noord (NL), Nieuwe Maas (NL), Nieuwe Waterweg (NL), Oude Maas (NL), Hartelkanaal (NL), Pannerdensch Kanaal (NL), Nederrijn (NL), Lek (NL)
# Donau (DE), Dunaj (SK), Duna (HU), Dunav (HR), Дунав (RS), Dunărea (RO), Дунав (BG), Дунай (UA)
# Canal de Bourbourg, Canal de la Colme, Aa, Canal d’Aire, Canal de la Deule, Canal de la Sensee (FR), Lys / Leie (FR, BE), Escaut / Scheldt / Schelde (FR, BE)
# Labe (CS), Elbe (DE) Vltava (CS), Moldau (DE) Weser (DE), Vezera (CS)
# Moselle (FR), Musel (LU), Mosel (DE)

countries = ['NL', 'BE', 'DE', 'FR', 'CH', 'BG', 'UA', 'HU', 'HR', 'SK', 'RO', 'CS', 'LU']

In [3]:
resp = requests.get(files_url.format(country_code='LU'), headers={"Accept": "application/json", 'User-Agent': 'test-agent'})
resp

<Response [200]>

In [8]:
path_re = re.compile(r'(?P<country>[A-Z]{2})_(?P<dataset>[\w]+)_(?P<date>[\d]+)_(?P<version>v\d+\.\d+)\.zip')


data = resp.json()
dfs = []
for country_code in countries:
    url = files_url.format(country_code=country_code)
    resp = requests.get(url, headers=headers)
    data = resp.json()
    for row in data:
        name = row['name']
        match = path_re.search(name)
        if match:
            row.update(match.groupdict())
    df_i = pd.DataFrame(data)
    dfs.append(df_i) 

files = pd.concat(dfs)
set(files['name'])

files


,name,countryCode,lastModified,fileSizeInB,country,dataset,date,version
0,NL_NetworkData_20230927_v2.4.zip,NL,2023-09-27T11:46:37+02:00,3384784,NL,NetworkData,20230927,v2.4
1,NL_NetworkData_20231011_v2.4.zip,NL,2023-10-12T00:00:19+02:00,3817798,NL,NetworkData,20231011,v2.4
2,NL_NetworkData_20231025_v2.4.zip,NL,2023-10-26T00:00:33+02:00,3793950,NL,NetworkData,20231025,v2.4
3,NL_NetworkData_20240417_v2.4.zip,NL,2024-04-18T00:00:09+02:00,2220,NL,NetworkData,20240417,v2.4
4,NL_NetworkData_20240429_v2.4.zip,NL,2024-04-29T10:54:50+02:00,3765799,NL,NetworkData,20240429,v2.4
...,...,...,...,...,...,...,...,...
4,LU_NetworkData_20240527_v2.4.zip,LU,2024-05-27T10:51:36+02:00,23676,LU,NetworkData,20240527,v2.4
5,LU_RisIndex_20230927_v3.0.zip,LU,2023-09-27T11:51:32+02:00,13314,LU,RisIndex,20230927,v3.0
6,LU_RisIndex_20240308_v3.0.zip,LU,2024-03-09T00:00:15+01:00,13355,LU,RisIndex,20240308,v3.0
7,LU_RisIndex_20240429_v3.0.zip,LU,2024-04-29T11:06:30+02:00,13353,LU,RisIndex,20240429,v3.0


In [10]:
last_files = files.sort_values(['countryCode', 'dataset', 'lastModified']).groupby(['countryCode', 'dataset']).last().reset_index()
last_files

,countryCode,dataset,name,lastModified,fileSizeInB,country,date,version
0,BE,NetworkData,BE_NetworkData_20240527_v2.4.zip,2024-05-27T10:51:34+02:00,1571189,BE,20240527,v2.4
1,BE,RisIndex,BE_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:16+02:00,866243,BE,20240527,v3.0
2,BG,NetworkData,BG_NetworkData_20240527_v2.4.zip,2024-05-27T10:51:37+02:00,64710,BG,20240527,v2.4
3,BG,RisIndex,BG_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,140174,BG,20240527,v3.0
4,DE,NetworkData,DE_NetworkData_20240527_v2.4.zip,2024-05-27T10:51:36+02:00,2293175,DE,20240527,v2.4
5,DE,RisIndex,DE_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,2282383,DE,20240527,v3.0
6,FR,NetworkData,FR_NetworkData_20240527_v2.4.zip,2024-05-27T10:51:40+02:00,3127704,FR,20240527,v2.4
7,FR,RisIndex,FR_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:18+02:00,3709902,FR,20240527,v3.0
8,HR,NetworkData,HR_NetworkData_20240527_v2.4.zip,2024-05-27T10:51:34+02:00,25585,HR,20240527,v2.4
9,HR,RisIndex,HR_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,116008,HR,20240527,v3.0


In [11]:
for i, row in last_files.iterrows():
    url = download_url.format(filename=row['name'])
    with contextlib.chdir(data_dir):
        !wget --content-disposition '{url}'
        !unzip '{row['name']}'
    

--2024-06-10 16:46:31--  https://www.eurisportal.eu/AWFImportData/api/ExportFile/DownloadFile?fileName=BE_NetworkData_20240527_v2.4.zip
Resolving www.eurisportal.eu (www.eurisportal.eu)... 137.116.212.234
Connecting to www.eurisportal.eu (www.eurisportal.eu)|137.116.212.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1571189 (1.5M) [application/zip]
Saving to: ‘BE_NetworkData_20240527_v2.4.zip’

BE_NetworkData_2024 100%[===================>]   1.50M  2.96MB/s    in 0.5s    

2024-06-10 16:46:31 (2.96 MB/s) - ‘BE_NetworkData_20240527_v2.4.zip’ saved [1571189/1571189]

Archive:  BE_NetworkData_20240527_v2.4.zip
  inflating: BerthArea_BE_20240527.geojson  
  inflating: Berth_BE_20240527.geojson  
  inflating: BridgeArea_BE_20240527.geojson  
  inflating: BridgeOpening_BE_20240527.geojson  
  inflating: FairwaySection_BE_20240527.geojson  
  inflating: LockChamberArea_BE_20240527.geojson  
  inflating: LockChamberPartArea_BE_20240527.geojson  
  inflating: Loc

In [12]:
!ls {data_dir}

BE_NetworkData_20240527_v2.4.zip     LockChamberArea_SK_20240527.geojson
BE_RisIndex_20240527_v3.0.zip	     LockChamberPartArea_BE_20240527.geojson
BG_NetworkData_20240527_v2.4.zip     LockChamberPart_BE_20240527.geojson
BG_RisIndex_20240527_v3.0.zip	     LockChamber_BE_20240527.geojson
BerthArea_BE_20240527.geojson	     LockChamber_DE_20240527.geojson
BerthArea_BG_20240527.geojson	     LockChamber_FR_20240527.geojson
BerthArea_DE_20240527.geojson	     LockChamber_LU_20240527.geojson
BerthArea_FR_20240527.geojson	     LockChamber_NL_20240527.geojson
BerthArea_HR_20240527.geojson	     LockChamber_RO_20240527.geojson
BerthArea_HU_20240527.geojson	     LockChamber_SK_20240527.geojson
BerthArea_LU_20240527.geojson	     LockComplex_BE_20240527.geojson
BerthArea_NL_20240527.geojson	     LockComplex_DE_20240527.geojson
BerthArea_RO_20240527.geojson	     LockComplex_FR_20240527.geojson
BerthArea_SK_20240527.geojson	     LockComplex_NL_20240527.geojson
Berth_BE_20240527.geojson	     LockComplex

In [9]:
!open {data_dir}